In [1]:
#libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine, euclidean, hamming
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import linear_kernel


In [2]:
#read the data & have randome sample for reproducability
recipes = pd.read_csv('../data/full_clean_data.csv')
recipes.sample(3, random_state=123)

,name,url,category,rating,rating_count,ingredients,total,servings,yield,calories,carbohydrates_g,fat_g,protein_g,ingredients_parsed
7358,Maple Dill Carrots,https://www.allrecipes.com/recipe/87694/maple-...,side-dish,4.57,729,"['3 cups peeled and sliced carrots ', ' 2 tabl...",20,4,4 servings,117.3,16.1,6.0,1.0,"['carrot', 'butter', 'sugar']"
2656,Dill Pickle Soup,https://www.allrecipes.com/recipe/54804/dill-p...,soups-stews-and-chili,4.31,205,"['2 tablespoons butter ', ' ½ cup all-purpose ...",30,8,8 servings,104.3,13.7,4.2,3.1,"['butter', 'flour', 'chicken broth', 'pickle',..."
12521,White Bean Tabbouleh,https://www.allrecipes.com/recipe/205611/white...,salad,4.74,46,"['1 cup bulgur wheat, uncooked ', ' 1\u2009½ c...",110,6,6 servings,197.2,24.8,9.7,5.6,"['bulgur', 'spinach', 'cannellini']"


In [3]:
##rename the dataframe id
recipes.index.names = ['recipes_id']
recipes.head(2)

,name,url,category,rating,rating_count,ingredients,total,servings,yield,calories,carbohydrates_g,fat_g,protein_g,ingredients_parsed
recipes_id,,,,,,,,,,,,,,
0,Dessert Crepes,https://www.allrecipes.com/recipe/19037/desser...,breakfast-and-brunch,4.80,1156,"['4 eggs, lightly beaten ', ' 1\u2009⅓ cups m...",20,8,8 crepes,163.8,17.2,7.7,6.4,"['milk', 'flour', 'sugar']"
1,Chicken Parmesan,https://www.allrecipes.com/recipe/223042/chick...,world-cuisine,4.83,4245,"['4 skinless, boneless chicken breast halves ...",60,4,4 servings,470.8,24.8,24.9,42.1,"['chicken breast', 'egg', 'panko bread', 'parm..."


In [4]:
##remove punctuation from parsed_ingredients?
recipes['ingredients_parsed'] = recipes['ingredients_parsed'].str.replace('[^\w\s]','')
recipes.head(2)

/tmp/ipykernel_17576/1542198556.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  recipes['ingredients_parsed'] = recipes['ingredients_parsed'].str.replace('[^\w\s]','')


,name,url,category,rating,rating_count,ingredients,total,servings,yield,calories,carbohydrates_g,fat_g,protein_g,ingredients_parsed
recipes_id,,,,,,,,,,,,,,
0,Dessert Crepes,https://www.allrecipes.com/recipe/19037/desser...,breakfast-and-brunch,4.80,1156,"['4 eggs, lightly beaten ', ' 1\u2009⅓ cups m...",20,8,8 crepes,163.8,17.2,7.7,6.4,milk flour sugar
1,Chicken Parmesan,https://www.allrecipes.com/recipe/223042/chick...,world-cuisine,4.83,4245,"['4 skinless, boneless chicken breast halves ...",60,4,4 servings,470.8,24.8,24.9,42.1,chicken breast egg panko bread parmesan cheese...


In [5]:
##normalize the numerical column (nutritional values ) in new dataframe << because i dont want to normalize all other features
newrecipes = recipes[['total', 'calories', 'carbohydrates_g', 'fat_g', 'protein_g']]
recipes_norm = pd.DataFrame(normalize(newrecipes, axis=0))
recipes_norm.columns = newrecipes.columns
recipes_norm.index = newrecipes.index
recipes_norm.head(2)

,total,calories,carbohydrates_g,fat_g,protein_g
recipes_id,,,,,
0,0.000837,0.003888,0.003539,0.003818,0.002726
1,0.002512,0.011174,0.005103,0.012346,0.017931


In [31]:

#####creare a function. Guidance reference https://www.kaggle.com/code/yyzz1010/content-based-filtering-recipe-recommender?scriptVersionId=21219821&cellId=22 
def selected_recipe(recipes_id):
    recipe_df = recipes.set_index('recipes_id')
    x = "{}  {}".format(recipes_id, recipe_df.at[recipes_id, 'name'])
    print(x)

In [40]:
#####creare a function. Guidance reference https://www.kaggle.com/code/yyzz1010/content-based-filtering-recipe-recommender?scriptVersionId=21219821&cellId=23
def recipe_recommender(distance_method, recipes_id, N):

    
     allRecipes = pd.DataFrame(recipes_norm.index)
     allRecipes = allRecipes[allRecipes.recipes_id != recipes_id]
     allRecipes["distance"] = allRecipes["recipes_id"].apply(lambda x: distance_method(recipes_norm.loc[recipes_id], recipes_norm.loc[x]))
     TopNRecommendation = allRecipes.sort_values(["distance"]).head(N).sort_values(by=['distance', 'recipes_id'])
     # sort by distance then recipe id, the smaller value of recipe id will be picked. 
    
     recipe_df = recipes.set_index('recipes_id')
     recipes_id = [recipes_id]
     recipe_list = []
     for recipeid in TopNRecommendation.recipes_id:
        recipes_id.append(recipeid)   # list of recipe id of selected recipe and recommended recipe
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'url']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'name']))
     print(recipe_list)
     return recipes_norm.loc[recipes_id, :]

In [32]:
selected_recipe(12521)

12521  White Bean Tabbouleh


In [41]:
recipe_recommender(cosine, 12521, 3)

['10633  https://www.allrecipes.com/recipe/244326/greek-farro-salad/', '10633  Greek Farro Salad', '13949  https://www.allrecipes.com/recipe/218884/uptown-cowboy-caviar/', '13949  Uptown Cowboy Caviar', '10643  https://www.allrecipes.com/recipe/232590/roasted-beet-and-kale-salad/', '10643  Roasted Beet and Kale Salad']


,total,calories,carbohydrates_g,fat_g,protein_g
recipes_id,,,,,
12521,0.004606,0.004680,0.005103,0.004809,0.002385
10633,0.007746,0.008226,0.008725,0.008577,0.004387
13949,0.006071,0.006309,0.006708,0.005999,0.003067
10643,0.005862,0.006233,0.007017,0.006346,0.002854


In [ ]:
##How can I take user input and add it this function?